In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
import pandas as pd
import torch
import pickle
import math
import prior.CustomDistribution as custom
import config
import time
import IPython.display as IPd
import warnings

from forward import Forward
from survey import Survey

from sbi import utils as utils
from sbi.utils import user_input_checks as uic

from util import *
from block_utils import *
from pygimli_utils import *
from polynomials import *

from sklearn.metrics import mean_squared_error

_ = torch.manual_seed(0)

2024-07-09 14:22:38.260784: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-09 14:22:38.300025: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-09 14:22:38.300056: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-09 14:22:38.300080: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-09 14:22:38.308334: I tensorflow/core/platform/cpu_feature_g

In [2]:
max_depth = 23.5
step_size = 0.5
num_points = int(max_depth / step_size) + 1
max_resistivity = 1000
min_resistivity = 1
num_measurements = 23
num_dim = 48

depths = np.linspace(0, max_depth, num=num_dim)
thicknesses = torch.ones(num_dim) * 0.5

In [3]:
# Survey Setup and Forward Model Setup

survey, electrode_spacing = Survey(num_measurements, min_ab_distance=2, max_ab_distance=100).schlumberger_survey()
forward = Forward(thicknesses, survey)

## Load all prior distributions

In [4]:
# **** Step prior
custom_prior = custom.CustomDistribution(low=min_resistivity, high=max_resistivity, dimension=num_dim)
prior_step, *_ = uic.process_prior(custom_prior,
                                   custom_prior_wrapper_kwargs=dict(
                                       lower_bound=torch.tensor([custom_prior.low] * custom_prior.dimension),
                                       upper_bound=torch.tensor([custom_prior.high] * custom_prior.dimension)
                                   )
                                   )

# **** Independent Step Prior
# Prior Setup
prior_min = torch.as_tensor([min_resistivity] * num_dim)
prior_max = torch.as_tensor([max_resistivity] * num_dim)
prior_ind_step = utils.BoxUniform(low=prior_min * torch.ones(num_dim), high=prior_max * torch.ones(num_dim))

# **** Block Prior 
n_blocks = 4
res_prior_min = torch.as_tensor([min_resistivity] * n_blocks)
res_prior_max = torch.as_tensor([max_resistivity] * n_blocks)

n_thickness = 3
min_thickness = 0.1
max_thickness = 20.0
thickness_prior_min = torch.as_tensor([min_thickness] * n_thickness)
thickness_prior_max = torch.as_tensor([max_thickness] * n_thickness)

prior_min = torch.cat([res_prior_min, thickness_prior_min])
prior_max = torch.cat([res_prior_max, thickness_prior_max])

prior_block = utils.BoxUniform(low=prior_min, high=prior_max)

# **** Polynomial Prior
num_dim_prior = 5
prior_min = torch.as_tensor([-1] * num_dim_prior)
prior_max = torch.as_tensor([1] * num_dim_prior)
prior_poly = utils.BoxUniform(low=prior_min * torch.ones(num_dim_prior), high=prior_max * torch.ones(num_dim_prior))


/opt/bwhpc/common/jupyter/tensorflow/2023-10-10/lib/python3.9/site-packages/torch/distributions/distribution.py:53: UserWarning: <class 'prior.CustomDistribution.CustomDistribution'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(


## Load all the helper functions and simulators for the models

#### Step Model

In [5]:
# Helper functions for the Step Model 

def steps_forward_model_no_noise(res):
    ''' Noiseless forward simulator for the step prior '''
    return forward.dc_forward_model(res)


def steps_forward_model_base_noise_5(res):
    ''' Forward simulator for the step prior adding white noise (of magnitude 5 \Omega m) to the simulations '''
    if (np.any([res < 0])):
        nan_array = np.empty((num_measurements,))
        nan_array[:] = np.nan
        return nan_array
    return forward.dc_forward_model_base_noise(res, 5)


def rmse(sample, gt):
    return np.sqrt(mean_squared_error(gt, sample))


def compute_avg_rmse_and_map_rsme_from_step_posterior_sammples(gt_res, gt_app_res):
    '''
    Computation of error metrics: 
    The posterior ist samples 10,000 times from which 1,000 samples are evaluated under the forward model. 
    Among the 1,000 posterior predictive samples the minimal RMSE is assessed. 
    The respective RMSE of the posterior sample that belongs to the minimal RMSE prediction is caluculated.
    The average error over all posterior samples and posterior predictive samples is computed. 

    For comparison with the minimal RMSEs the MAP sample and it's prediction is computed and the RMSEs are calculated.

    returns all error metrics
    '''

    posterior_sample_size = 10_000
    simulation_sample_size = 1_000
    percentiles = config.percentiles

    posterior_step.set_default_x(gt_app_res)
    # Sample from Step posterior
    posterior_samples_step = posterior_step.sample((posterior_sample_size,), show_progress_bars=True)

    # Get random samples form the posterior samples
    perm = torch.randperm(posterior_sample_size)
    idx = perm[:simulation_sample_size]
    random_post_samples = posterior_samples_step[idx]

    # Simulate the Apparent Resistivity from the samples
    sim_apparent_resistivities = []
    min_rmse_ar = np.infty
    min_rmse_res = np.infty

    # Compute the minimal RMSE of the posterior predictive samples
    for sample in random_post_samples:
        app_res = steps_forward_model_no_noise(np.array(sample))
        rmse_value_ar = rmse(app_res, gt_app_res)

        if rmse_value_ar <= min_rmse_ar:
            min_rmse_ar = rmse_value_ar
            min_rmse_res = rmse(sample, gt_res)
        sim_apparent_resistivities.append(app_res)

    print("Min RMSE is: ", min_rmse_ar)
    mean_res = np.nanmean(random_post_samples, axis=0)
    mean_ar = np.nanmean(sim_apparent_resistivities, axis=0)

    avg_rmse_param = avg_rmse(gt_res, random_post_samples)
    avg_rmse_obs = avg_rmse(gt_app_res, sim_apparent_resistivities)

    # Compute the MAP RMSE in obsrevation and parameter space
    map = posterior_step.map()[0]
    map_app_res = steps_forward_model_no_noise(np.array(map))

    map_rmse_res = rmse(map, gt_res)
    map_rmse_app_res = rmse(map_app_res, gt_app_res)

    return min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map, map_app_res, map_rmse_res, map_rmse_app_res

#### Polynomial Model

In [6]:
polynomial = Polynomial(max_depth=max_depth, num_dim=num_dim, max_res=max_resistivity,
                        num_measurements=num_measurements)


def legendre_forward_nan(coefficients):
    ''' 
    Forward Simulator for the Polynomial Prior 
    Polymomials that exceed the range of [-1,1] are excluded by outputting an array of nan values. 
    This is to ensure that samples are within the well-defined range of Legendre Polynoms.  

    retturns the apparent resistivity signal from the polynomial depth profile
    '''
    poly = polynomial.legendre_polynomial_nan(coefficients)
    if np.any((poly < -1) | (poly > 1)):
        return np.full((num_measurements,), np.nan)
    else:
        # Shift from [-1,1] interval to [0,2]
        # Then multiply by max_resistivity/2 value to cover the resistivity value range [0, max_resistivity]
        res = polynomial.poly_to_resistivity(poly)
        return forward.dc_forward_model(res)


# Interpolate Legendre polynomial coefficients and remove invalid simulations by substitue with nan values
def legendre_forward_nan_noise_5(coefficients):
    ''' 
    Forward Simulator for the Polynomial Prior that includes white noise (of magnitude 5 \Omega m) to the simulations
    Polymomials that exceed the range of [-1,1] are excluded by outputting an array of nan values. 
    This is to ensure that samples are within the well-defined range of Legendre Polynoms.  

    returns the apparent resistivity signal from the polynomial depth profile
    '''
    poly = polynomial.legendre_polynomial_nan(coefficients)
    if np.any((poly < -1) | (poly > 1)):
        return np.full((num_measurements,), np.nan)
    else:
        # Shift from [-1,1] interval to [0,2]
        # Then multiply by max_resistivity/2 value to cover the resistivity value range [0, max_resistivity]
        res = polynomial.poly_to_resistivity(poly)
        return forward.dc_forward_model_base_noise(res, 5)


def compute_avg_rmse_and_map_rsme_from_poly_posterior_sammples(gt_res, gt_app_res):
    '''
    Computation of error metrics: 
    The posterior is sampled 10,000 times from which 1,000 samples are evaluated under the forward model. 
    Among the 1,000 posterior predictive samples the minimal RMSE is assessed. 
    The respective RMSE of the posterior sample that belongs to the minimal RMSE prediction is caluculated.
    The average error over all posterior samples and posterior predictive samples is computed. 

    For comparison with the minimal RMSEs the MAP sample and it's prediction is computed and the RMSEs are calculated.

    returns all error metrics
    '''

    posterior_sample_size = 10_000
    simulation_sample_size = 1_000
    percentiles = config.percentiles

    posterior_poly.set_default_x(gt_app_res)
    # Sample from Step posterior
    posterior_samples_poly = posterior_poly.sample((posterior_sample_size,), show_progress_bars=True)

    # Get random samples form the posterior samples
    perm = torch.randperm(posterior_sample_size)
    idx = perm[:simulation_sample_size]
    random_post_samples = posterior_samples_poly[idx]

    # Simulate the Apparent Resistivity from the samples
    post_resistivities = []
    sim_apparent_resistivities = []
    min_rmse_ar = np.infty
    min_rmse_res = np.infty

    # Compute the minimal RMSE of the posterior predictive samples
    for sample in random_post_samples:
        app_res = legendre_forward_nan(np.array(sample))
        if np.isnan(app_res).all():
            continue

        post_resistivities.append(polynomial.coefficients_to_resistivity(sample))
        rmse_value_ar = rmse(app_res, gt_app_res)

        if rmse_value_ar <= min_rmse_ar:
            min_rmse_ar = rmse_value_ar
            min_rmse_res = rmse(polynomial.coefficients_to_resistivity(sample), gt_res)
        sim_apparent_resistivities.append(app_res)

    print("Min RMSE is: ", min_rmse_ar)
    mean_res = np.nanmean(random_post_samples, axis=0)
    mean_ar = np.nanmean(sim_apparent_resistivities, axis=0)

    avg_rmse_param = avg_rmse(gt_res, post_resistivities)
    avg_rmse_obs = avg_rmse(gt_app_res, sim_apparent_resistivities)

    # Compute the MAP RMSE in observation and parameter space check for potential invalid MAP estimate that exceeds the polynomial inverval of [-1,1]
    map = posterior_poly.map()[0]
    res_map = polynomial.coefficients_to_resistivity(map)

    if not np.isnan(res_map).all():
        map_rmse_res = rmse(polynomial.coefficients_to_resistivity(map), gt_res)
    else:
        map_rmse_res = np.nan

    map_app_res = legendre_forward_nan(np.array(map))
    if not np.isnan(map_app_res).all():
        map_rmse_app_res = rmse(map_app_res, gt_app_res)
    else:
        map_rmse_app_res = np.nan

    return min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map, map_app_res, map_rmse_res, map_rmse_app_res


#### Independent Step Model

In [13]:
def compute_avg_rmse_and_map_rsme_from_ind_step_posterior_sammples(gt_res, gt_app_res):
    '''
    Computation of error metrics: 
    The posterior is sampled 2,000 times from which 1,000 samples are evaluated under the forward model. 
    I sampled only 2,000 times due to the longer simulation time of that prior, as some of the 
    Among the 1,000 posterior predictive samples the minimal RMSE is assessed. 
    The respective RMSE of the posterior sample that belongs to the minimal RMSE prediction is caluculated.
    The average error over all posterior samples and posterior predictive samples is computed. 

    For comparison with the minimal RMSEs the MAP sample and it's prediction is computed and the RMSEs are calculated.

    returns all error metrics
    '''

    posterior_sample_size = 2_000
    simulation_sample_size = 1_000

    posterior_ind_step.set_default_x(gt_app_res)
    # Sample from Step posterior
    posterior_samples_ind_step = posterior_ind_step.sample((posterior_sample_size,), show_progress_bars=True)

    # Get random samples form the posterior samples
    perm = torch.randperm(posterior_sample_size)
    idx = perm[:simulation_sample_size]
    random_post_samples = posterior_samples_ind_step[idx]

    # Simulate the Apparent Resistivity from the samples
    sim_apparent_resistivities = []
    min_rmse_ar = np.infty
    min_rmse_res = np.infty

    # Compute the minimal RMSE of the posterior predictive samples
    for sample in random_post_samples:
        app_res = steps_forward_model_no_noise(np.array(sample))
        rmse_value_ar = rmse(app_res, gt_app_res)

        if rmse_value_ar <= min_rmse_ar:
            min_rmse_ar = rmse_value_ar
            min_rmse_res = rmse(sample, gt_res)
        sim_apparent_resistivities.append(app_res)

    print("Min RMSE is: ", min_rmse_ar)
    mean_res = np.nanmean(random_post_samples, axis=0)
    mean_ar = np.nanmean(sim_apparent_resistivities, axis=0)

    avg_rmse_param = avg_rmse(gt_res, random_post_samples)
    avg_rmse_obs = avg_rmse(gt_app_res, sim_apparent_resistivities)

    # Compute the MAP RMSE in obsrevation and parameter space
    map = posterior_ind_step.map()[0]
    map_app_res = steps_forward_model_no_noise(np.array(map))

    map_rmse_res = rmse(map, gt_res)
    map_rmse_app_res = rmse(map_app_res, gt_app_res)

    return min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map, map_app_res, map_rmse_res, map_rmse_app_res


#### Block Model

In [12]:
def split_params(res_n_depth):
    ''' 
    Split a block sample into its resistivity values and the layer thicknesses
    '''
    number_of_layers = int((len(res_n_depth) + 1) / 2)

    # First half of parameters is resistivities
    resistivities = res_n_depth[:number_of_layers]

    # Second half of parameters are thicknesses
    thicknesses = res_n_depth[number_of_layers:]
    return resistivities, thicknesses


def thickness_to_depth(t):
    ''' 
    Convert the individual layer thicknesses to a depth profile as the layers are stacked on top of each other
    '''
    last_layer_tickness = torch.tensor([1.0])
    thickness_to_depth = np.cumsum(torch.cat((t, last_layer_tickness)))
    return thickness_to_depth


def dc_forward_block_noise(res_n_depth):
    '''
    Simulator used for generating the training set (noise level 5 \Omega m)
    returns the apparent resistiviy values from  a block sample. 
    '''
    resistivities, thicknesses = split_params(res_n_depth)
    forward = Forward(thicknesses, survey)

    return forward.dc_forward_model_base_noise(resistivities, 5)


def dc_forward_block_no_noise(res_n_depth):
    '''
    Noiseless Simulator
    returns the apparent resistiviy values from  a block sample. 
    '''
    resistivities, thicknesses = split_params(res_n_depth)
    forward = Forward(thicknesses, survey)

    return forward.dc_forward_model(resistivities)


def ensure_48_points(block_res):
    '''    
    Check if the depth profile has 48 values. If depth profile is too short, extend the last layers -> used for plotting
    in case the model has 48 points it is not modified. If it has less, the resistivity values of the last layer are appenden to ensure 48 points.
    return block resistivities
    '''

    # Check if the array has less than 48 values
    while len(block_res) < 48:
        # Append the last value to the array until it has 48 values
        block_res.append(block_res[-1])
    return np.array(block_res)


def block_sample_to_resistivity(sample):
    '''
    Transforms a block prior or posterior sample into the resistivity depth profile
    returns the 
    '''
    inv_res_thick_depth, max_depth = transfrom_and_find_max_reached_depth([sample])
    mapped_sample = map_resistivities_to_depth(inv_res_thick_depth, max_depth)
    low_res_sample = transform_to_lower_res_samples(mapped_sample)[0]
    return ensure_48_points(low_res_sample)


def compute_avg_rmse_and_map_rsme_from_block_posterior_sammples(gt_res, gt_app_res):
    '''
    Computation of error metrics: 
    The posterior ist samples 10,000 times from which 1,000 samples are evaluated under the forward model. 
    Among the 1,000 posterior predictive samples the minimal RMSE is assessed. 
    The respective RMSE of the posterior sample that belongs to the minimal RMSE prediction is caluculated.
    The average error over all posterior samples and posterior predictive samples is computed. 

    For comparison with the minimal RMSEs the MAP sample and it's prediction is computed and the RMSEs are calculated.

    returns all error metrics
    '''

    posterior_sample_size = 10_000
    simulation_sample_size = 1_000
    percentiles = config.percentiles

    posterior_block.set_default_x(gt_app_res)
    # Sample from Step posterior
    posterior_samples_block = posterior_block.sample((posterior_sample_size,), show_progress_bars=True)

    # Get random samples form the posterior samples
    perm = torch.randperm(posterior_sample_size)
    idx = perm[:simulation_sample_size]
    random_post_samples = posterior_samples_block[idx]

    # Simulate the Apparent Resistivity from the samples
    post_resistivities = []
    sim_apparent_resistivities = []
    min_rmse_ar = np.infty
    min_rmse_res = np.infty

    for sample in random_post_samples:
        app_res = dc_forward_block_no_noise(np.array(sample))
        res = block_sample_to_resistivity(sample)
        rmse_value_ar = rmse(app_res, gt_app_res)

        if rmse_value_ar <= min_rmse_ar:
            min_rmse_ar = rmse_value_ar
            min_rmse_res = rmse(res, gt_res)
        sim_apparent_resistivities.append(app_res)
        post_resistivities.append(res)

    print("Min RMSE is: ", min_rmse_ar)
    mean_res = np.nanmean(post_resistivities, axis=0)
    mean_ar = np.nanmean(sim_apparent_resistivities, axis=0)

    avg_rmse_param = avg_rmse(gt_res, post_resistivities)
    avg_rmse_obs = avg_rmse(gt_app_res, sim_apparent_resistivities)

    # Compute the MAP RMSE in observation and parameter space 
    map = posterior_block.map()[0]
    map_app_res = dc_forward_block_no_noise(np.array(map))

    map_rmse_res = rmse(block_sample_to_resistivity(map), gt_res)
    map_rmse_app_res = rmse(map_app_res, gt_app_res)

    return min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map, map_app_res, map_rmse_res, map_rmse_app_res

## Load all posterior models

In [9]:
# **** Step prior
filename_step_model = "pickled_objects/steps_100k_1000_ohm_48points_new.pkl"
inference_step, posterior_step = unpickle_objects(filename_step_model)

# **** Polynomial Prior
filename_legendre_model = "pickled_objects/legendre_100k_1000_ohm_48points_new.pkl"
inference_poly, posterior_poly = unpickle_objects(filename_legendre_model)

# **** Independent Step Prior
filename_ind_steps_model = "pickled_objects/ind_steps_500k_1000_ohm_48points.pkl"
inference_ind_step, posterior_ind_step = unpickle_objects(filename_ind_steps_model)

# **** Block Prior 
filename_block_model = "pickled_objects/block_100k_1000_ohm_48points.pkl"
inference_block, posterior_block = unpickle_objects(filename_block_model)

## Generate 10 samples for each prior

In [10]:
filename_step = 'data/prior_samples/step_samples.pkl'
filename_poly = 'data/prior_samples/poly_samples.pkl'
filename_block = 'data/prior_samples/block_samples.pkl'
filename_ind_step = 'data/prior_samples/ind_step_samples.pkl'

generate_samples = False

if generate_samples:
    num_samples = 10

    prior_samples_step = prior_step.sample((num_samples,))
    prior_samples_poly = prior_poly.sample(
        (num_samples * num_samples,))  # Here we sample more to ensure that at least 10 valid samples are contained
    prior_samples_block = prior_block.sample((num_samples,))
    prior_samples_ind_step = prior_ind_step.sample((num_samples,))

    # Step Prior Predictive Samples
    app_res_samples_step = []
    for sample in prior_samples_step:
        app_res_sample = steps_forward_model_base_noise_5(sample)
        app_res_samples_step.append(app_res_sample)

    # Poly Prior Predictive Samples
    app_res_samples_poly = []
    valid_prior_samples_poly = []
    valid_samples = 0
    for sample in prior_samples_poly:
        app_res_sample = legendre_forward_nan_noise_5(sample)
        if not np.isnan(app_res_sample).all():
            app_res_samples_poly.append(app_res_sample)
            valid_prior_samples_poly.append(sample)
            valid_samples += 1
        if valid_samples >= num_samples:
            break
    prior_samples_poly = valid_prior_samples_poly

    # Block Prior Predictive Samples
    app_res_samples_block = []
    for sample in prior_samples_block:
        app_res_sample = dc_forward_block_noise(sample)
        app_res_samples_block.append(app_res_sample)

    # Ind. Step Predictive Samples
    app_res_samples_ind_step = []
    for sample in prior_samples_ind_step:
        app_res_sample = steps_forward_model_base_noise_5(sample)
        app_res_samples_ind_step.append(app_res_sample)

    pickle_objects([app_res_samples_step, prior_samples_step], filename=filename_step)
    pickle_objects([app_res_samples_poly, prior_samples_poly], filename=filename_poly)
    pickle_objects([app_res_samples_block, prior_samples_block], filename=filename_block)
    pickle_objects([app_res_samples_ind_step, prior_samples_ind_step], filename=filename_ind_step)

else:

    (app_res_samples_step, prior_samples_step) = unpickle_objects(filename_step)
    (app_res_samples_poly, prior_samples_poly) = unpickle_objects(filename_poly)
    (app_res_samples_block, prior_samples_block) = unpickle_objects(filename_block)
    (app_res_samples_ind_step, prior_samples_ind_step) = unpickle_objects(filename_ind_step)

## Evaluate the samples on the different posterior models

### Step model 

In [61]:
# On Step Samples

rmse_info_step_on_step = []
for sample, ar_sample in zip(prior_samples_step, app_res_samples_step):
    (min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map, map_app_res, map_rmse_res,
     map_rmse_app_res) = compute_avg_rmse_and_map_rsme_from_step_posterior_sammples(sample, ar_sample)
    rmse_info_step_on_step.append((min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map,
                                   map_app_res, map_rmse_res, map_rmse_app_res))

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  1.9724422359627791


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  2.3239031224858087


...

Min RMSE is:  2.792332521625606


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  1.786514764438931


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

In [78]:
min_rsme_values_res = [item[0] for item in rmse_info_step_on_step]
min_rsme_values_app_res = [item[1] for item in rmse_info_step_on_step]
print("min_rsme_values_res", np.average(min_rsme_values_res))
print("min_rsme_values_app_res", np.average(min_rsme_values_app_res))

min_rsme_values_res 2.2951506169301275
min_rsme_values_app_res 81.00451


In [17]:
# On Poly Samples

rmse_info_step_on_poly = []
for sample, ar_sample in zip(prior_samples_poly, app_res_samples_poly):
    res_sample = polynomial.coefficients_to_resistivity(sample)

    (min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map, map_app_res, map_rmse_res,
     map_rmse_app_res) = compute_avg_rmse_and_map_rsme_from_step_posterior_sammples(res_sample, ar_sample)
    rmse_info_step_on_poly.append((min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map,
                                   map_app_res, map_rmse_res, map_rmse_app_res))



Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  2.506885448810622


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  2.9683222965760416


...

Min RMSE is:  2.8925932290840946


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  3.1371094293451987


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

In [18]:
min_rsme_values_res = [item[0] for item in rmse_info_step_on_poly]
min_rsme_values_app_res = [item[1] for item in rmse_info_step_on_poly]
print("min_rsme_values_res", np.average(min_rsme_values_res))
print("min_rsme_values_app_res", np.average(min_rsme_values_app_res))

min_rsme_values_res 2.842490053875064
min_rsme_values_app_res 83.45431352177198


In [19]:
# On Block Samples

rmse_info_step_on_block = []
for sample, ar_sample in zip(prior_samples_block, app_res_samples_block):
    res_sample = block_sample_to_resistivity(sample)
    (min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map, map_app_res, map_rmse_res,
     map_rmse_app_res) = compute_avg_rmse_and_map_rsme_from_step_posterior_sammples(res_sample, ar_sample)
    rmse_info_step_on_block.append((min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map,
                                    map_app_res, map_rmse_res, map_rmse_app_res))

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  29.04259438496528


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  2.106472468897799


...

Min RMSE is:  3.742570296138903


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  2.7225978298836506


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

In [20]:
min_rsme_values_res = [item[0] for item in rmse_info_step_on_block]
min_rsme_values_app_res = [item[1] for item in rmse_info_step_on_block]
print("min_rsme_values_res", np.average(min_rsme_values_res))
print("min_rsme_values_app_res", np.average(min_rsme_values_app_res))

min_rsme_values_res 5.848679528856853
min_rsme_values_app_res 152.68843084255238


In [79]:
# On Ind. Step Samples

rmse_info_step_ind_step = []
for sample, ar_sample in zip(prior_samples_ind_step, app_res_samples_ind_step):
    (min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map, map_app_res, map_rmse_res,
     map_rmse_app_res) = compute_avg_rmse_and_map_rsme_from_step_posterior_sammples(sample, ar_sample)
    rmse_info_step_ind_step.append((min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map,
                                    map_app_res, map_rmse_res, map_rmse_app_res))

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  5.093022360917463


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  3.380941176810242


...

Min RMSE is:  3.9923593116448117


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  3.3587686282302918


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

In [81]:
min_rsme_values_res = [item[0] for item in rmse_info_step_ind_step]
min_rsme_values_app_res = [item[1] for item in rmse_info_step_ind_step]
print("min_rsme_values_res", np.average(min_rsme_values_res))
print("min_rsme_values_app_res", np.average(min_rsme_values_app_res))

min_rsme_values_res 4.05583805008063
min_rsme_values_app_res 345.066


In [ ]:
pickle_objects([rmse_info_step_on_step, rmse_info_step_on_poly, rmse_info_step_on_block, rmse_info_step_ind_step],
               filename='data/error metrics/errors_step_on_priors.pkl')

### Polynomial model 

In [93]:
# On Step Samples

rmse_info_poly_on_step = []
for sample, ar_sample in zip(prior_samples_step, app_res_samples_step):
    (min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map, map_app_res, map_rmse_res,
     map_rmse_app_res) = compute_avg_rmse_and_map_rsme_from_poly_posterior_sammples(sample, ar_sample)
    rmse_info_poly_on_step.append((min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map,
                                   map_app_res, map_rmse_res, map_rmse_app_res))

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  4.786009491932953


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  9.136195359180151


...

Min RMSE is:  8.14913430185609


Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  4.30202425747349


Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  5.878157814451509


In [94]:
min_rsme_values_res = [item[0] for item in rmse_info_poly_on_step]
min_rsme_values_app_res = [item[1] for item in rmse_info_poly_on_step]
print("min_rsme_values_res", np.average(min_rsme_values_res))
print("min_rsme_values_app_res", np.average(min_rsme_values_app_res))

min_rsme_values_res 5.648778770035522
min_rsme_values_app_res 352.8436562524123


In [24]:
# On Poly Samples
rmse_info_poly_on_poly = []
for sample, ar_sample in zip(prior_samples_poly, app_res_samples_poly):
    res_sample = polynomial.coefficients_to_resistivity(sample)

    (min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map, map_app_res, map_rmse_res,
     map_rmse_app_res) = compute_avg_rmse_and_map_rsme_from_poly_posterior_sammples(res_sample, ar_sample)
    rmse_info_poly_on_poly.append((min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map,
                                   map_app_res, map_rmse_res, map_rmse_app_res))



Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  2.6299309462541083


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  2.839270274394389


...

Min RMSE is:  2.453308308937231


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  2.5751339571282013


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

In [25]:
min_rsme_values_res = [item[0] for item in rmse_info_poly_on_poly]
min_rsme_values_app_res = [item[1] for item in rmse_info_poly_on_poly]
print("min_rsme_values_res", np.average(min_rsme_values_res))
print("min_rsme_values_app_res", np.average(min_rsme_values_app_res))

min_rsme_values_res 2.5267486516193665
min_rsme_values_app_res 11.359500231433396


In [30]:
# On Block Samples

rmse_info_poly_on_block = []
for sample, ar_sample in zip(prior_samples_block[1:], app_res_samples_block[1:]):
    res_sample = block_sample_to_resistivity(sample)
    (min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map, map_app_res, map_rmse_res,
     map_rmse_app_res) = compute_avg_rmse_and_map_rsme_from_poly_posterior_sammples(res_sample, ar_sample)
    rmse_info_poly_on_block.append((min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map,
                                    map_app_res, map_rmse_res, map_rmse_app_res))



Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  2.018621505261236


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  5.225007141541075


...

Min RMSE is:  6.503997579892688


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  2.5684114858484004


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

In [34]:
min_rsme_values_res = [item[0] for item in rmse_info_poly_on_block]
min_rsme_values_app_res = [item[1] for item in rmse_info_poly_on_block]

# Filter out inf values
filtered_min_rsme_values_res = [value for value in min_rsme_values_res if np.isfinite(value)]
filtered_min_rsme_values_app_res = [value for value in min_rsme_values_app_res if np.isfinite(value)]

average_res = np.average(filtered_min_rsme_values_res)
average_app_res = np.average(filtered_min_rsme_values_app_res)

print("min_rsme_values_res", average_res)
print("min_rsme_values_app_res", average_app_res)

min_rsme_values_res 3.506435545563648
min_rsme_values_app_res 123.7869615744539


In [63]:
# Polynomial Posterior on Independent Step Prior Samples
valid_prior_samples = torch.cat([prior_samples_ind_step[:6], prior_samples_ind_step[7:]])
valid_app_res_samples = torch.cat(
    [torch.tensor(app_res_samples_ind_step)[:6], torch.tensor(app_res_samples_ind_step)[7:]])

rmse_info_poly_on_ind_step = []
for sample, ar_sample in zip(valid_prior_samples, valid_app_res_samples):
    (min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map, map_app_res, map_rmse_res,
     map_rmse_app_res) = compute_avg_rmse_and_map_rsme_from_poly_posterior_sammples(sample, ar_sample)
    rmse_info_poly_on_ind_step.append((min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map,
                                       map_app_res, map_rmse_res, map_rmse_app_res))

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  9.239259770815378


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  2.982997401082521


...

Min RMSE is:  2.3957981463459856


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  4.835089236381552


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

In [67]:
min_rsme_values_res = [item[0] for item in rmse_info_poly_on_ind_step]
min_rsme_values_app_res = [item[1] for item in rmse_info_poly_on_ind_step]

# Filter out inf values
filtered_min_rsme_values_res = [value for value in min_rsme_values_res if np.isfinite(value)]
filtered_min_rsme_values_app_res = [value for value in min_rsme_values_app_res if np.isfinite(value)]

average_res = np.average(filtered_min_rsme_values_res)
average_app_res = np.average(filtered_min_rsme_values_app_res)

print("min_rsme_values_res", np.average(average_res))
print("min_rsme_values_app_res", np.average(average_app_res))

min_rsme_values_res 6.866947254482145
min_rsme_values_app_res 338.19755347616865


In [ ]:
pickle_objects([rmse_info_poly_on_step, rmse_info_poly_on_poly, rmse_info_poly_on_block, rmse_info_poly_on_ind_step],
               filename='data/error metrics/errors_poly_on_priors.pkl')

### Independent Step model 

In [15]:
# On Step Samples
valid_prior_samples = torch.cat([prior_samples_step[:1], prior_samples_step[2:]])
valid_app_res_samples = torch.cat([torch.tensor(app_res_samples_step)[:1], torch.tensor(app_res_samples_step)[2:]])

rmse_info_ind_step_on_step = []
for sample, ar_sample in zip(valid_prior_samples, valid_app_res_samples):
    (min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map, map_app_res, map_rmse_res,
     map_rmse_app_res) = compute_avg_rmse_and_map_rsme_from_ind_step_posterior_sammples(sample, ar_sample)
    rmse_info_ind_step_on_step.append((min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map,
                                       map_app_res, map_rmse_res, map_rmse_app_res))

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  7.849544389778869


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

                    accepted. It may take a long time to collect the remaining
                    798 samples. Consider interrupting (Ctrl-C) and switching to
                    `build_posterior(..., sample_with='mcmc')`.


Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

...

Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

                    accepted. It may take a long time to collect the remaining
                    906 samples. Consider interrupting (Ctrl-C) and switching to
                    `build_posterior(..., sample_with='mcmc')`.


Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  2.594455129422203


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

In [17]:
min_rsme_values_res = [item[0] for item in rmse_info_ind_step_on_step]
min_rsme_values_app_res = [item[1] for item in rmse_info_ind_step_on_step]
print("min_rsme_values_res", np.average(min_rsme_values_res))
print("min_rsme_values_app_res", np.average(min_rsme_values_app_res))

min_rsme_values_res 8.450122140688055
min_rsme_values_app_res 223.77444


In [14]:
# On Poly Samples

rmse_info_ind_step_on_poly = []
for sample, ar_sample in zip(prior_samples_poly, app_res_samples_poly):
    res_sample = polynomial.coefficients_to_resistivity(sample)

    (min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map, map_app_res, map_rmse_res,
     map_rmse_app_res) = compute_avg_rmse_and_map_rsme_from_ind_step_posterior_sammples(res_sample, ar_sample)
    rmse_info_ind_step_on_poly.append((min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map,
                                       map_app_res, map_rmse_res, map_rmse_app_res))



Drawing 2000 posterior samples:   0%|          | 0/2000 [00:00<?, ?it/s]

                    accepted. It may take a long time to collect the remaining
                    1986 samples. Consider interrupting (Ctrl-C) and switching to
                    `build_posterior(..., sample_with='mcmc')`.


Min RMSE is:  29.408758442821153


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

                    accepted. It may take a long time to collect the remaining
                    976 samples. Consider interrupting (Ctrl-C) and switching to
                    `build_posterior(..., sample_with='mcmc')`.


...

Min RMSE is:  3.667742017140337


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 2000 posterior samples:   0%|          | 0/2000 [00:00<?, ?it/s]

Min RMSE is:  2.7576516248165595


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

In [15]:
min_rsme_values_res = [item[0] for item in rmse_info_ind_step_on_poly]
min_rsme_values_app_res = [item[1] for item in rmse_info_ind_step_on_poly]
print("min_rsme_values_res", np.average(min_rsme_values_res))
print("min_rsme_values_app_res", np.average(min_rsme_values_app_res))

min_rsme_values_res 7.981827363807135
min_rsme_values_app_res 272.08384924798116


In [42]:
# On Block Samples
valid_prior_samples = torch.cat([prior_samples_block[1:2], prior_samples_block[3:8], prior_samples_block[9:]])
valid_app_res_samples = torch.cat([torch.tensor(app_res_samples_block)[1:2], torch.tensor(app_res_samples_block)[3:8],
                                   torch.tensor(app_res_samples_block[9:])])

rmse_info_ind_step_on_block = []
for sample, ar_sample in zip(valid_prior_samples, valid_app_res_samples):
    res_sample = block_sample_to_resistivity(sample)
    (min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map, map_app_res, map_rmse_res,
     map_rmse_app_res) = compute_avg_rmse_and_map_rsme_from_ind_step_posterior_sammples(res_sample, ar_sample)
    rmse_info_ind_step_on_block.append((min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map,
                                        map_app_res, map_rmse_res, map_rmse_app_res))

Drawing 2000 posterior samples:   0%|          | 0/2000 [00:00<?, ?it/s]

                    accepted. It may take a long time to collect the remaining
                    1986 samples. Consider interrupting (Ctrl-C) and switching to
                    `build_posterior(..., sample_with='mcmc')`.


Min RMSE is:  9.70564613746275


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

                    accepted. It may take a long time to collect the remaining
                    906 samples. Consider interrupting (Ctrl-C) and switching to
                    `build_posterior(..., sample_with='mcmc')`.


...

Min RMSE is:  3.1016290495305974


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 2000 posterior samples:   0%|          | 0/2000 [00:00<?, ?it/s]

Min RMSE is:  3.448198944311309


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

In [43]:
min_rsme_values_res = [item[0] for item in rmse_info_ind_step_on_block]
min_rsme_values_app_res = [item[1] for item in rmse_info_ind_step_on_block]
print("min_rsme_values_res", np.average(min_rsme_values_res))
print("min_rsme_values_app_res", np.average(min_rsme_values_app_res))

min_rsme_values_res 6.326088864995233
min_rsme_values_app_res 286.38997998893234


In [29]:
# On Ind. Step Samples

rmse_info_ind_step_ind_step = []
for sample, ar_sample in zip(prior_samples_ind_step, app_res_samples_ind_step):
    (min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map, map_app_res, map_rmse_res,
     map_rmse_app_res) = compute_avg_rmse_and_map_rsme_from_ind_step_posterior_sammples(sample, ar_sample)
    rmse_info_ind_step_ind_step.append((min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map,
                                        map_app_res, map_rmse_res, map_rmse_app_res))

Drawing 2000 posterior samples:   0%|          | 0/2000 [00:00<?, ?it/s]

Min RMSE is:  2.9024632049080994


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 2000 posterior samples:   0%|          | 0/2000 [00:00<?, ?it/s]

Min RMSE is:  2.667670029501797


...

Min RMSE is:  2.6915365905785147


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 2000 posterior samples:   0%|          | 0/2000 [00:00<?, ?it/s]

Min RMSE is:  3.9742653331027946


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

In [30]:
min_rsme_values_res = [item[0] for item in rmse_info_ind_step_ind_step]
min_rsme_values_app_res = [item[1] for item in rmse_info_ind_step_ind_step]
print("min_rsme_values_res", np.average(min_rsme_values_res))
print("min_rsme_values_app_res", np.average(min_rsme_values_app_res))

min_rsme_values_res 3.1937416000563132
min_rsme_values_app_res 353.29193


In [ ]:
pickle_objects(
    [rmse_info_ind_step_on_step, rmse_info_ind_step_on_poly, rmse_info_ind_step_on_block, rmse_info_ind_step_ind_step],
    filename='data/error metrics/errors_ind_step_on_priors.pkl')

### Block model 

In [43]:
# On Step Samples
rmse_info_block_on_step = []
for sample, ar_sample in zip(prior_samples_step, app_res_samples_step):
    (min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map, map_app_res, map_rmse_res,
     map_rmse_app_res) = compute_avg_rmse_and_map_rsme_from_block_posterior_sammples(sample, ar_sample)
    rmse_info_block_on_step.append((min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map,
                                    map_app_res, map_rmse_res, map_rmse_app_res))

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  2.9516167575715953


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  6.603228813510669


...

Min RMSE is:  2.9167194608700315


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  5.969925061037695


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

In [51]:
min_rsme_values_res = [item[0] for item in rmse_info_block_on_step]
min_rsme_values_app_res = [item[1] for item in rmse_info_block_on_step]
print("min_rsme_values_res", np.average(min_rsme_values_res))
print("min_rsme_values_app_res", np.average(min_rsme_values_app_res))

min_rsme_values_res 3.7609370471257635
min_rsme_values_app_res 89.01716894259746


In [66]:
# On Poly Samples
rmse_info_block_on_poly = []
for sample, ar_sample in zip(prior_samples_poly, app_res_samples_poly):
    res_sample = polynomial.coefficients_to_resistivity(sample)
    if np.isnan(res_sample).all():
        continue
    (min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map, map_app_res, map_rmse_res,
     map_rmse_app_res) = compute_avg_rmse_and_map_rsme_from_block_posterior_sammples(res_sample, ar_sample)
    rmse_info_block_on_poly.append((min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map,
                                    map_app_res, map_rmse_res, map_rmse_app_res))

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  3.2756799443464564


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  4.423874470572068


...

Min RMSE is:  2.8556023302612434


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  6.090041292403803


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

In [68]:
min_rsme_values_res = [item[0] for item in rmse_info_block_on_poly]
min_rsme_values_app_res = [item[1] for item in rmse_info_block_on_poly]
print("min_rsme_values_res", np.average(min_rsme_values_res))
print("min_rsme_values_app_res", np.average(min_rsme_values_app_res))

min_rsme_values_res 3.6454709975938173
min_rsme_values_app_res 123.59375411532582


In [58]:
# On Block Samples

rmse_info_block_on_block = []
for sample, ar_sample in zip(prior_samples_block, app_res_samples_block):
    res_sample = block_sample_to_resistivity(sample)
    (min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map, map_app_res, map_rmse_res,
     map_rmse_app_res) = compute_avg_rmse_and_map_rsme_from_block_posterior_sammples(res_sample, ar_sample)
    rmse_info_block_on_block.append((min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map,
                                     map_app_res, map_rmse_res, map_rmse_app_res))

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  2.848925143737222


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  2.251742716754679


...

Min RMSE is:  2.554595339730582


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  2.520642850442397


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

In [59]:
min_rsme_values_res = [item[0] for item in rmse_info_block_on_block]
min_rsme_values_app_res = [item[1] for item in rmse_info_block_on_block]
print("min_rsme_values_res", np.average(min_rsme_values_res))
print("min_rsme_values_app_res", np.average(min_rsme_values_app_res))

min_rsme_values_res 2.4931129450791274
min_rsme_values_app_res 77.59322627536916


In [61]:
# On Ind. Step Samples

rmse_info_block_on_ind_step = []
for sample, ar_sample in zip(prior_samples_ind_step, app_res_samples_ind_step):
    (min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map, map_app_res, map_rmse_res,
     map_rmse_app_res) = compute_avg_rmse_and_map_rsme_from_block_posterior_sammples(sample, ar_sample)
    rmse_info_block_on_ind_step.append((min_rmse_ar, min_rmse_res, mean_res, mean_ar, avg_rmse_param, avg_rmse_obs, map,
                                        map_app_res, map_rmse_res, map_rmse_app_res))

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  3.157172948450546


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  4.136704626422415


...

Min RMSE is:  2.4313318298576494


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

Min RMSE is:  6.155211913719567


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

In [62]:
min_rsme_values_res = [item[0] for item in rmse_info_block_on_ind_step]
min_rsme_values_app_res = [item[1] for item in rmse_info_block_on_ind_step]
print("min_rsme_values_res", np.average(min_rsme_values_res))
print("min_rsme_values_app_res", np.average(min_rsme_values_app_res))

min_rsme_values_res 7.460923828101296
min_rsme_values_app_res 335.06598124415814


In [ ]:
pickle_objects(
    [rmse_info_block_on_step, rmse_info_block_on_poly, rmse_info_block_on_block, rmse_info_block_on_ind_step],
    filename='data/error metrics/errors_block_on_priors.pkl')